In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import DBSCAN
from collections import Counter
import mysql.connector
from mysql.connector import Error
import datetime # Adăugat pentru a gestiona corect NOW() și datele evenimentelor

# --- Configurația Bazei de Date ---
DB_CONFIG = {
    'host': 'localhost',
    'database': 'artmapdb',
    'user': 'root',
    'password': 'RootPass123#'
}

# --- Funcții Utilitare pentru Baza de Date ---
def create_db_connection():
    """Creează o conexiune la baza de date MySQL."""
    connection = None
    try:
        connection = mysql.connector.connect(**DB_CONFIG)
        print("Conexiune la MySQL DB reușită!")
    except Error as e:
        print(f"Eroare la conectare: '{e}'")
    return connection

def execute_query(connection, query, params=None):
    """Execută o interogare și returnează rezultatele ca DataFrame."""
    cursor = connection.cursor()
    try:
        cursor.execute(query, params)
        if cursor.description: # Verifică dacă interogarea a produs un set de rezultate (ex. SELECT)
            result = cursor.fetchall()
            columns = [i[0] for i in cursor.description]
            return pd.DataFrame(result, columns=columns)
        else: # Pentru interogări care nu returnează date (ex. INSERT, UPDATE, DELETE)
            connection.commit() 
            return pd.DataFrame() 
    except Error as e:
        print(f"Eroare la executarea interogării: '{e}'")
        return pd.DataFrame()
    finally:
        cursor.close()

def clear_table_data(connection, table_name):
    """Șterge toate datele dintr-un tabel specific."""
    if connection is None or not connection.is_connected():
        print(f"Eroare: Conexiunea la baza de date nu este validă pentru ștergerea datelor din {table_name}.")
        return False
    
    cursor = connection.cursor()
    query = f"DELETE FROM {table_name};"
    try:
        cursor.execute(query)
        connection.commit()
        print(f"Toate datele din tabelul '{table_name}' au fost șterse (tranzacție comitată).")
        return True
    except Error as e:
        connection.rollback()
        print(f"Eroare la ștergerea datelor din tabelul '{table_name}': '{e}' (tranzacție anulată).")
        return False
    finally:
        cursor.close()

def save_clusters_to_db(connection, combined_features_df):
    """Salvează rezultatele clusterizării utilizatorilor în baza de date."""
    if connection is None or not connection.is_connected():
        print("Eroare: Conexiunea la baza de date nu este validă pentru salvarea clusterelor.")
        return

    cursor = connection.cursor()
    data_to_insert = []
    for user_id, row in combined_features_df.iterrows():
        cluster_id = row['cluster']
        try:
            user_id_int = int(user_id)
        except ValueError:
            print(f"Skipping user_id '{user_id}' due to conversion error to int.")
            continue
        data_to_insert.append((user_id_int, int(cluster_id)))

    if not data_to_insert:
        print("Nu există date de clusterizare pentru a fi salvate.")
        return

    try:
        sql_insert = """
        INSERT INTO user_clusters (user_id, cluster_id)
        VALUES (%s, %s);
        """
        cursor.executemany(sql_insert, data_to_insert)
        connection.commit()
        print(f"Clusterizările pentru {len(data_to_insert)} utilizatori au fost salvate în baza de date (tranzacție comitată).")
    except Error as e:
        connection.rollback()
        print(f"Eroare la salvarea clusterelor în DB: '{e}' (tranzacție anulată).")
    finally:
        cursor.close()

def save_recommendations_to_db(connection, user_id, recommendations):
    """Salvează recomandările pentru un utilizator în baza de date."""
    if connection is None or not connection.is_connected():
        print("Eroare: Conexiunea la baza de date nu este validă pentru salvarea recomandărilor.")
        return

    cursor = connection.cursor()
    try:
        data_to_insert = []
        for i, rec in enumerate(recommendations):
            event_date_val = rec.get('date')
            if isinstance(event_date_val, (datetime.datetime, datetime.date)):
                event_date_str = event_date_val.strftime('%Y-%m-%d %H:%M:%S')
            elif event_date_val is None:
                event_date_str = None
            else: 
                event_date_str = str(event_date_val) if event_date_val else None

            data_to_insert.append((
                int(user_id), 
                int(rec['id']), 
                float(rec['match_score']),
                int(i + 1), # rank
                event_date_str 
            ))
        
        if not data_to_insert:
            return

        sql_insert = """
        INSERT INTO user_recommendations (user_id, event_id, match_score, rank_order, event_date, recommended_at)
        VALUES (%s, %s, %s, %s, %s, NOW());
        """
        cursor.executemany(sql_insert, data_to_insert)
        connection.commit()
    except Error as e:
        connection.rollback()
        print(f"Eroare la salvarea recomandărilor pentru {user_id} în DB: '{e}' (tranzacție anulată).")
    except Exception as e: 
        connection.rollback()
        print(f"Eroare generală la salvarea recomandărilor pentru {user_id}: '{e}' (tranzacție anulată).")
    finally:
        cursor.close()

def recommend_events_for_user(user_id, user_clusters_df, user_prefs_df, events_df, event_genres_df, categories_df, genres_df, top_n=5, debug=False):
    if debug: print(f"\nGenerare recomandări pentru utilizatorul {user_id}")

    try:
        user_id_lookup = int(user_id)
    except ValueError:
        if debug: print(f"User ID '{user_id}' nu este un întreg valid.")
        return []
        
    if user_id_lookup not in user_clusters_df.index:
        if debug: print(f"Utilizatorul {user_id_lookup} nu a fost găsit în DataFrame-ul de clustere.")
        return []

    user_cluster = user_clusters_df.loc[user_id_lookup, 'cluster']
    if debug: print(f"Utilizatorul {user_id_lookup} aparține clusterului {user_cluster}")

    cluster_users_ids = user_clusters_df[user_clusters_df['cluster'] == user_cluster].index.tolist()
    if debug: print(f"Utilizatori din același cluster: {cluster_users_ids}")

    cluster_prefs_df = user_prefs_df[user_prefs_df['user_id'].isin(cluster_users_ids)]
    
    if cluster_prefs_df.empty:
        if debug: print(f"Nicio preferință găsită pentru utilizatorii din clusterul {user_cluster}.")
        return []

    popular_genres_counter = Counter(cluster_prefs_df['genre_id']).most_common()
    popular_categories_counter = Counter(cluster_prefs_df['category_id']).most_common()

    if debug:
        print(f"Genuri populare în cluster: {popular_genres_counter[:5]}")
        print(f"Categorii populare în cluster: {popular_categories_counter[:5]}")

    recommended_events_list = []
    recommended_event_ids_set = set()

    # Strategia 1: Potrivire exactă gen + categorie
    for genre_id, _ in popular_genres_counter:
        if genre_id == 0: continue 
        for category_id, _ in popular_categories_counter:
            if category_id == 0: continue 
            
            events_in_category_df = events_df[events_df['category_id'] == category_id]
            
            for _, event_row in events_in_category_df.iterrows():
                event_id = event_row['id']
                if genre_id in event_genres_df[event_genres_df['event_id'] == event_id]['genre_id'].values:
                    if event_id not in recommended_event_ids_set:
                        category_name = categories_df[categories_df['id'] == category_id]['name'].values[0] if category_id in categories_df['id'].values else 'N/A'
                        genre_name = genres_df[genres_df['id'] == genre_id]['name'].values[0] if genre_id in genres_df['id'].values else 'N/A'
                        
                        recommended_events_list.append({
                            'id': event_id, 'title': event_row['title'],
                            'description': event_row.get('description', 'Fără descriere'),
                            'category_id': category_id, 'category_name': category_name,
                            'genre_id': genre_id, 'genre_name': genre_name,
                            'match_score': 1.0,
                            'cheapest_ticket': event_row['cheapest_ticket'],
                            'date': event_row['date'], 'location': event_row['location']
                        })
                        recommended_event_ids_set.add(event_id)
                if len(recommended_events_list) >= top_n: break
            if len(recommended_events_list) >= top_n: break
        if len(recommended_events_list) >= top_n: break
    
    # Strategia 2: Potrivire doar pe categorie
    if len(recommended_events_list) < top_n:
        if debug: print("Adăugăm potriviri doar pe categorie...")
        for category_id, _ in popular_categories_counter:
            if category_id == 0: continue
            events_in_category_df = events_df[events_df['category_id'] == category_id]
            for _, event_row in events_in_category_df.iterrows():
                event_id = event_row['id']
                if event_id not in recommended_event_ids_set:
                    category_name = categories_df[categories_df['id'] == category_id]['name'].values[0] if category_id in categories_df['id'].values else 'N/A'
                    
                    genre_id_val = None
                    genre_name_val = 'N/A'
                    category_info = categories_df[categories_df['id'] == category_id]
                    if not category_info.empty and category_info['has_genre'].values[0] == 1:
                        event_genre_ids = event_genres_df[event_genres_df['event_id'] == event_id]['genre_id'].tolist()
                        for g_id, _ in popular_genres_counter:
                            if g_id == 0: continue
                            if g_id in event_genre_ids:
                                genre_id_val = g_id
                                genre_name_val = genres_df[genres_df['id'] == g_id]['name'].values[0] if g_id in genres_df['id'].values else 'N/A'
                                break
                    
                    recommended_events_list.append({
                        'id': event_id, 'title': event_row['title'],
                        'description': event_row.get('description', 'Fără descriere'),
                        'category_id': category_id, 'category_name': category_name,
                        'genre_id': genre_id_val, 'genre_name': genre_name_val,
                        'match_score': 0.7,
                        'cheapest_ticket': event_row['cheapest_ticket'],
                        'date': event_row['date'], 'location': event_row['location']
                    })
                    recommended_event_ids_set.add(event_id)
                if len(recommended_events_list) >= top_n: break
            if len(recommended_events_list) >= top_n: break

    # Strategia 3: Potrivire doar pe gen
    if len(recommended_events_list) < top_n:
        if debug: print("Adăugăm potriviri doar pe gen...")
        for genre_id, _ in popular_genres_counter:
            if genre_id == 0: continue
            event_ids_with_genre = event_genres_df[event_genres_df['genre_id'] == genre_id]['event_id'].unique()
            
            for event_id in event_ids_with_genre:
                if event_id not in recommended_event_ids_set:
                    event_details_series = events_df[events_df['id'] == event_id].iloc[0] if event_id in events_df['id'].values else None
                    if event_details_series is not None:
                        genre_name = genres_df[genres_df['id'] == genre_id]['name'].values[0] if genre_id in genres_df['id'].values else 'N/A'
                        category_id_val = event_details_series['category_id']
                        category_name = categories_df[categories_df['id'] == category_id_val]['name'].values[0] if category_id_val in categories_df['id'].values else 'N/A'

                        recommended_events_list.append({
                            'id': event_id, 'title': event_details_series['title'],
                            'description': event_details_series.get('description', 'Fără descriere'),
                            'category_id': category_id_val, 'category_name': category_name,
                            'genre_id': genre_id, 'genre_name': genre_name,
                            'match_score': 0.5,
                            'cheapest_ticket': event_details_series['cheapest_ticket'],
                            'date': event_details_series['date'], 'location': event_details_series['location']
                        })
                        recommended_event_ids_set.add(event_id)
                if len(recommended_events_list) >= top_n: break
            if len(recommended_events_list) >= top_n: break
            
    recommended_events_list.sort(key=lambda x: x['match_score'], reverse=True)
    if debug: print(f"Număr final de recomandări pentru {user_id_lookup}: {len(recommended_events_list[:top_n])}")
    return recommended_events_list[:top_n]

# --- NOU: Funcția de evaluare (separată pentru claritate) ---
def evaluate_recommendation_system(user_clusters_df, user_prefs_df, events_df, event_genres_df, categories_df, genres_df):
    """Evaluează sistemul de recomandare și generează vizualizări."""
    print("\nInițiere evaluare sistem de recomandare...")
    
    results_list = []
    # Iterează prin toți utilizatorii care au fost clusterizați
    for user_id in user_clusters_df.index:
        recommendations = recommend_events_for_user(
            user_id,
            user_clusters_df,
            user_prefs_df,
            events_df,
            event_genres_df,
            categories_df,
            genres_df,
            top_n=10, # Poți ajusta acest număr dacă e necesar pentru evaluare
            debug=False # De obicei False pentru evaluarea în masă
        )
        results_list.append({
            'user_id': user_id,
            'cluster': user_clusters_df.loc[user_id, 'cluster'],
            'num_recommendations': len(recommendations),
            'avg_match_score': np.mean([rec['match_score'] for rec in recommendations]) if recommendations else 0
        })
    
    if not results_list:
        print("Nu s-au putut colecta date pentru evaluare.")
        return

    evaluation_df = pd.DataFrame(results_list)
    print("\nSumarul Evaluării Sistemului de Recomandare:")
    print(evaluation_df.describe())

    cluster_eval_df = evaluation_df.groupby('cluster')[['num_recommendations', 'avg_match_score']].mean()
    print("\nRecomandări medii și scor de potrivire mediu pe cluster:")
    print(cluster_eval_df)

    # Generare vizualizări
    try:
        plt.figure(figsize=(12, 6))
        sns.barplot(x='cluster', y='num_recommendations', data=evaluation_df, palette="viridis", hue='cluster', legend=False)
        plt.title('Numărul Mediu de Recomandări pe Cluster')
        plt.xlabel('Cluster')
        plt.ylabel('Număr Mediu de Recomandări')
        plt.tight_layout()
        plt.savefig('recommendations_by_cluster.png')
        plt.close()
        print("- recommendations_by_cluster.png generat.")

        plt.figure(figsize=(12, 6))
        sns.barplot(x='cluster', y='avg_match_score', data=evaluation_df, palette="mako", hue='cluster', legend=False)
        plt.title('Scorul Mediu de Potrivire pe Cluster')
        plt.xlabel('Cluster')
        plt.ylabel('Scor Mediu de Potrivire')
        plt.tight_layout()
        plt.savefig('match_scores_by_cluster.png')
        plt.close()
        print("- match_scores_by_cluster.png generat.")
        
        print("\nAnaliza vizuală a sistemului de recomandare finalizată!")
        plt.show()
    except Exception as e:
        print(f"Eroare la generarea graficelor: {e}")


def main():
    print("Pornire script de clusterizare și recomandare...")
    connection = create_db_connection()

    if connection is None:
        print("Nu s-a putut stabili conexiunea la bază de date. Se oprește scriptul.")
        return

    # --- Pasul 1: Golirea tabelelor user_clusters și user_recommendations ---
    print("\nȘtergerea datelor existente din tabelele de output...")
    if not clear_table_data(connection, "user_clusters"):
        print("Eroare la golirea tabelului user_clusters. Se oprește scriptul.")
        if connection and connection.is_connected(): connection.close()
        return
    if not clear_table_data(connection, "user_recommendations"):
        print("Eroare la golirea tabelului user_recommendations. Se oprește scriptul.")
        if connection and connection.is_connected(): connection.close()
        return

    # --- Pasul 2: Încărcarea Datelor din Baza de Date ---
    print("\nÎncărcarea datelor din bază de date...")
    user_prefs_query = "SELECT preference_id, category_id, genre_id, user_id FROM user_preferences"
    user_prefs = execute_query(connection, user_prefs_query)

    event_genres_query = "SELECT event_id, genre_id FROM event_genres"
    event_genres = execute_query(connection, event_genres_query)

    events_query = "SELECT id, address, cheapest_ticket, created_at, date, description, latitude, longitude, location, title, updated_at, category_id, created_by FROM events"
    events = execute_query(connection, events_query)

    categories_query = "SELECT id, name, has_genre FROM categories"
    categories = execute_query(connection, categories_query)

    genres_query = "SELECT id, name, category_id FROM genres"
    genres = execute_query(connection, genres_query)

    if user_prefs.empty or events.empty or event_genres.empty or categories.empty or genres.empty:
        print("\nEroare: Unul sau mai multe DataFrames încărcate din bază de date sunt goale.")
        print(f"User preferences gol: {user_prefs.empty}")
        print(f"Events gol: {events.empty}")
        print(f"Event genres gol: {event_genres.empty}")
        print(f"Categories gol: {categories.empty}")
        print(f"Genres gol: {genres.empty}")
        if connection and connection.is_connected():
            connection.close()
            print("Conexiunea MySQL a fost închisă (din cauza erorii de date).")
        return

    # --- Pasul 3: Preprocesarea Datelor ---
    print("\nPreprocesarea datelor...")
    user_prefs['user_id'] = user_prefs['user_id'].astype(int)
    user_prefs['genre_id'] = user_prefs['genre_id'].fillna(0).astype(int) 
    user_prefs['category_id'] = user_prefs['category_id'].fillna(0).astype(int)

    event_genres['event_id'] = event_genres['event_id'].astype(int)
    event_genres['genre_id'] = event_genres['genre_id'].astype(int)

    events['id'] = events['id'].astype(int)
    events['category_id'] = events['category_id'].astype(int)
    events['date'] = pd.to_datetime(events['date'], errors='coerce')

    # --- Pasul 4: Crearea Matricei de Caracteristici și Clusterizarea ---
    print("\nCrearea matricei de caracteristici pentru utilizatori...")
    if user_prefs.empty:
        print("DataFrame-ul user_prefs este gol. Nu se poate crea matricea de caracteristici.")
        if connection and connection.is_connected(): connection.close()
        return

    user_genre_matrix = user_prefs.pivot_table(index='user_id', columns='genre_id', aggfunc='size', fill_value=0)
    user_category_matrix = user_prefs.pivot_table(index='user_id', columns='category_id', aggfunc='size', fill_value=0)
    
    user_genre_matrix.columns = [f'genre_{c}' for c in user_genre_matrix.columns]
    user_category_matrix.columns = [f'category_{c}' for c in user_category_matrix.columns]
    
    combined_features = pd.concat([user_genre_matrix, user_category_matrix], axis=1)

    if combined_features.empty:
        print("\nEroare: Matricea de caracteristici combinată este goală. Verifică datele din user_preferences.")
        if connection and connection.is_connected(): connection.close()
        return
    
    print("\nAplicarea clusterizării DBSCAN...")
    X = combined_features.values
    db = DBSCAN(eps=0.5, min_samples=2).fit(X) 
    labels = db.labels_
    combined_features['cluster'] = labels
    print(f"Număr de clustere descoperite (inclusiv zgomot -1): {len(np.unique(labels))}")
    print(f"Distribuția utilizatorilor pe clustere: {Counter(labels)}")

    # --- Pasul 5: Salvarea Clusterelor în Baza de Date ---
    if connection and connection.is_connected():
        print("\nSalvarea clusterelor în baza de date...")
        save_clusters_to_db(connection, combined_features[combined_features['cluster'] != -1]) # Opțional: exclude zgomotul
    else:
        print("Conexiunea la baza de date nu este disponibilă pentru salvarea clusterelor.")
        if connection and connection.is_connected(): connection.close()
        return 

    # --- Pasul 6: Generarea și Salvarea Recomandărilor pentru TOȚI Utilizatorii ---
    print("\nGenerarea și salvarea recomandărilor pentru toți utilizatorii clusterizați...")
    if 'cluster' in combined_features.columns and not combined_features.index.empty:
        users_to_recommend_for = combined_features.index.unique().tolist()
        
        total_users = len(users_to_recommend_for)
        print(f"Se vor genera recomandări pentru {total_users} utilizatori.")
        
        processed_count = 0
        for user_id_iter in users_to_recommend_for:
            if not (connection and connection.is_connected()):
                print("Conexiunea la DB a fost pierdută. Se încearcă reconectarea...")
                connection = create_db_connection()
                if not (connection and connection.is_connected()):
                    print("Reconectarea a eșuat. Se oprește procesul de recomandare.")
                    break 
            
            recommendations = recommend_events_for_user(
                user_id_iter, combined_features, user_prefs,
                events, event_genres, categories, genres,
                top_n=10, debug=False 
            )
            if recommendations:
                save_recommendations_to_db(connection, user_id_iter, recommendations)
            
            processed_count += 1
            if processed_count % 100 == 0 or processed_count == total_users:
                print(f"Procesat {processed_count}/{total_users} utilizatori pentru recomandări.")
        
        print(f"Finalizat generarea și salvarea recomandărilor pentru {processed_count} utilizatori.")
    else:
        print("Nu s-au putut genera recomandări (date de clusterizare invalide sau lipsă utilizatori).")
        
    # --- Pasul 7: Evaluare și Vizualizare ---
    if not combined_features.empty and 'cluster' in combined_features.columns:
        evaluate_recommendation_system(
            combined_features, 
            user_prefs, 
            events, 
            event_genres, 
            categories, 
            genres
        )
    else:
        print("Nu există suficiente date pentru evaluare (matricea de caracteristici este goală sau clusterizarea a eșuat).")

    # --- Finalizare ---
    if connection and connection.is_connected():
        connection.close()
        print("\nConexiunea MySQL a fost închisă.")
    print("Script finalizat.")

if __name__ == '__main__':
    main()

Pornire script de clusterizare și recomandare...
Conexiune la MySQL DB reușită!

Ștergerea datelor existente din tabelele de output...
Toate datele din tabelul 'user_clusters' au fost șterse (tranzacție comitată).
Toate datele din tabelul 'user_recommendations' au fost șterse (tranzacție comitată).

Încărcarea datelor din bază de date...

Preprocesarea datelor...

Crearea matricei de caracteristici pentru utilizatori...

Aplicarea clusterizării DBSCAN...
Număr de clustere descoperite (inclusiv zgomot -1): 1
Distribuția utilizatorilor pe clustere: Counter({np.int64(-1): 5})

Salvarea clusterelor în baza de date...
Nu există date de clusterizare pentru a fi salvate.

Generarea și salvarea recomandărilor pentru toți utilizatorii clusterizați...
Se vor genera recomandări pentru 5 utilizatori.
Procesat 5/5 utilizatori pentru recomandări.
Finalizat generarea și salvarea recomandărilor pentru 5 utilizatori.

Inițiere evaluare sistem de recomandare...

Sumarul Evaluării Sistemului de Recomanda